In [2]:
#
# Import Package
import os
import shutil
import gc
import time
import random as rn
import numpy as np
import pandas as pd
import warnings
import csv

import scipy.io as sio
from scipy import signal
from tqdm import tqdm
import matplotlib.pyplot as plt

from scipy import sparse
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold

''' '''
# from resnet_ecg.utils import one_hot,get_batches
from resnet_ecg.ecg_preprocess import ecg_preprocessing
from resnet_ecg.densemodel import Net
from keras.utils import to_categorical
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
import keras.backend as K
from keras.layers import Input
from keras.models import Model, load_model
import keras
import pywt


warnings.filterwarnings("ignore")

'''
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = tf.Session(config=config)
KTF.set_session(session)
'''
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
tf.set_random_seed(1234)

# path of training data
path = '/media/jdcloud/'#"/media/uuser/data/finals/"#'/media/jdcloud/'

Using TensorFlow backend.


In [6]:
class Config(object):
    def __init__(self):
        self.conv_subsample_lengths = [1, 2, 1, 2, 1, 2, 1, 2]
        self.conv_filter_length = 32
        self.conv_num_filters_start = 12
        self.conv_init = "he_normal"
        self.conv_activation = "relu"
        self.conv_dropout = 0.5
        self.conv_num_skip = 2
        self.conv_increase_channels_at = 2
        self.batch_size = 32  # 128
        self.input_shape = [2560, 12]  # [1280, 1]
        self.num_categories = 2

    @staticmethod
    def lr_schedule(epoch):
        lr = 0.1
        if epoch >= 10 and epoch < 20:
            lr = 0.01
        if epoch >= 20:
            lr = 0.001
        # print('Learning rate: ', lr)
        return lr

def wavelet(ecg, wavefunc, lv, m, n):  #

    coeff = pywt.wavedec(ecg, wavefunc, mode='sym', level=lv)  #
    # sgn = lambda x: 1 if x > 0 else -1 if x < 0 else 0

    for i in range(m, n + 1):
        cD = coeff[i]
        for j in range(len(cD)):
            Tr = np.sqrt(2 * np.log(len(cD)))
            if cD[j] >= Tr:
                coeff[i][j] = np.sign(cD[j]) - Tr
            else:
                coeff[i][j] = 0

    denoised_ecg = pywt.waverec(coeff, wavefunc)
    return denoised_ecg


def wavelet_db6(sig):
    """
    R J, Acharya U R, Min L C. ECG beat classification using PCA, LDA, ICA and discrete
     wavelet transform[J].Biomedical Signal Processing and Control, 2013, 8(5): 437-448.
    param sig: 1-D numpy Array
    return: 1-D numpy Array
    """
    coeffs = pywt.wavedec(sig, 'db6', level=9)
    coeffs[-1] = np.zeros(len(coeffs[-1]))
    coeffs[-2] = np.zeros(len(coeffs[-2]))
    coeffs[0] = np.zeros(len(coeffs[0]))
    sig_filt = pywt.waverec(coeffs, 'db6')
    return sig_filt


def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    # Calculates the F score, the weighted harmonic mean of precision and recall.
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    # Calculates the f-measure, the harmonic mean of precision and recall.
    return fbeta_score(y_true, y_pred, beta=1)


def read_data_seg(data_path, split="Train", preprocess=False, fs=500, newFs=256, winSecond=10, winNum=10, n_index=0,pre_type="sym"):
    """ Read data """

    # Fixed params
    # n_index = 0
    n_class = 9
    winSize = winSecond * fs
    new_winSize = winSecond * newFs
    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    # label_path = os.path.join(data_path, "reference.txt")
    # labels = pd.read_csv(label_path, sep='\t',header = None)
    # labels = pd.read_csv("reference.csv")

    # Read time-series data
    channel_files = os.listdir(path_signals)
    # print(channel_files)
    channel_files.sort()
    n_channels = 12  # len(channel_files)
    # posix = len(split) + 5

    # Initiate array
    list_of_channels = []

    X = np.zeros((len(channel_files), new_winSize, n_channels)).astype('float32')
    i_ch = 0

    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1', 'II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II', 'aVR', 'V2', 'V5']
    channel_post_name = ['III', 'aVF', 'V3', 'V6']

    for i_ch, fil_ch in enumerate(channel_files[:]):  # tqdm

        if i_ch % 1000 == 0:
            print(i_ch)

        ecg = sio.loadmat(os.path.join(path_signals, fil_ch))
        ecg_length = ecg["I"].shape[1]

        if ecg_length > fs * winNum * winSecond:
            print(" too long !!!", ecg_length)
            ecg_length = fs * winNum * winSecond
        if ecg_length < 4500:
            print(" too short !!!", ecg_length)
            break

        slide_steps = int((ecg_length - winSize) / winSecond)

        if ecg_length <= 4500:
            slide_steps = 0

        ecg_channels = np.zeros((new_winSize, n_channels)).astype('float32')

        for i_n, ch_name in enumerate(channel_name):

            ecg_channels[:, i_n] = signal.resample(ecg[ch_name]
                                                   [:, n_index * slide_steps:n_index * slide_steps + winSize].T
                                                   , new_winSize).T
            if preprocess:
                if pre_type == "sym":
                    ecg_channels[:, i_n] = ecg_preprocessing(ecg_channels[:, i_n].reshape(1, new_winSize), 'sym8', 8, 3,
                                                             newFs, removebaseline=False, normalize=False)[0]
                elif pre_type == "db4":
                    ecg_channels[:, i_n] = wavelet(ecg_channels[:, i_n], 'db4', 4, 2, 4)
                elif pre_type == "db6":
                    ecg_channels[:, i_n] = wavelet_db6(ecg_channels[:, i_n])

                # ecg_channels[:, i_n] = (ecg_channels[:, i_n]-np.mean(ecg_channels[:, i_n]))/np.std(ecg_channels[:, i_n])
            else:
                pass
                print(" no preprocess !!! ")

        X[i_ch, :, :] = ecg_channels

    return X


def preprocess_y(labels, y, num_class=10):
    bin_label = np.zeros((len(y), num_class)).astype('int8')
    for i in range(len(y)):
        label_nona = labels.loc[y[i]].dropna()
        for j in range(1, label_nona.shape[0]):
            bin_label[i, int(label_nona[j])] = 1
    return bin_label

class DataGenerator(keras.utils.Sequence):
    # ' Generates data for Keras '

    def __init__(self, list_IDs, labels, quarter_labels,batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        # 'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.quarter_labels = quarter_labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        # 'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # 'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        # 'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # 'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size,  *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            IDsplit = ID.split("_")
            if IDsplit[0] == "quarter":
                X[i,] = np.load("/media/uuser/data/ysecgfinals/training_data/" + ID.split("_")[1] + ".npy")

                y[i,:] = preprocess_y(self.quarter_labels,self.quarter_labels[self.quarter_labels["File_name"] == IDsplit[0]+"_"+IDsplit[1]].index)

            else:
                # Store sample
                X[i,] = np.load("training_data/" + ID+".npy")
                # Store class
                y[i,:] = preprocess_y(self.labels,self.labels[self.labels["File_name"] == ID.split("_")[0]].index)

        # X_list = [(X[:, i]-np.mean(X[:, i]))/np.std(X[:, i]) for i in range(10)]
        X_list = [X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], X[:, 6], X[:, 7], X[:, 8], X[:, 9]]
        del X

        return X_list, y  # keras.utils.to_categorical(y, num_classes=self.n_classes)


def add_compile(model, config):
    optimizer = SGD(lr=config.lr_schedule(0), momentum=0.9)  # Adam()#
    model.compile(loss='binary_crossentropy',  # weighted_loss,#'binary_crossentropy',
                  optimizer='adam',  # optimizer,#'adam',
                  metrics=['accuracy', fmeasure, precision])  # recall
    # ['accuracy',fbetaMacro,recallMacro,precisionMacro])
    # ['accuracy',fmeasure,recall,precision])

In [8]:
train_dataset_path = path + "/Train/"
val_dataset_path = path + "/Val/"

train_files = os.listdir(train_dataset_path)
train_files.sort()
val_files = os.listdir(val_dataset_path)
val_files.sort()

labels = pd.read_csv(path + "REFERENCE.csv")
labels_en = pd.read_csv(path + "kfold_labels_en.csv")
#data_info = pd.read_csv(path + "data_info.csv")

input_size = (2560, 12)
net_num = 10
inputs_list = [Input(shape=input_size) for _ in range(net_num)]
net = Net()
outputs = net.nnet(inputs_list, 0.5, num_classes=10, attention=False)
model = Model(inputs=inputs_list, outputs=outputs)
# print(model.summary())

raw_IDs = labels_en["File_name"].values.tolist()

for j in range(4):
    for ids in labels[labels.label1==4]["File_name"]:
        if ids in raw_IDs:
            raw_IDs.append(ids)

for j in range(2):
    for ids in labels[labels.label1==7]["File_name"]:
        if ids in raw_IDs:
            raw_IDs.append(ids)

for j in range(1):
    for ids in labels[labels.label1==9]["File_name"]:
        if ids in raw_IDs:
            raw_IDs.append(ids)
                        
extend_db4_IDs = [i + "_db4" for i in raw_IDs]
extend_db6_IDs = [i + "_db6" for i in raw_IDs]
all_IDs = raw_IDs + extend_db4_IDs + extend_db6_IDs

train_labels = labels_en["label1"].values
all_train_labels = np.hstack((train_labels, train_labels, train_labels))

# Parameters
params = {'dim': (10, 2560),
          'batch_size': 64,
          'n_classes': 10,
          'n_channels': 12,
          'shuffle': True}

en_amount = 1
model_path = './official_densenet_model/'
index = np.arange(27453)#20067
np.random.shuffle(index)

index_train = index[:27453]
index_valid = index[14046:]

tr_IDs = np.array(all_IDs)[index_train]
val_IDs = np.array(all_IDs)[index_valid]

print(tr_IDs.shape)
print(val_IDs.shape)

# Generators
training_generator = DataGenerator(tr_IDs, labels,quarter_labels=None, **params)
#validation_generator = DataGenerator(val_IDs, labels, **params)

checkpointer = ModelCheckpoint(filepath=model_path + 'densenet_extend_weights-best_one_fold_all_data.hdf5',
                               monitor='val_fmeasure', verbose=1, save_best_only=True,
                               save_weights_only=True,
                               mode='max')  # val_fmeasure
reduce = ReduceLROnPlateau(monitor='val_fmeasure', factor=0.5, patience=2, verbose=1, min_delta=1e-4,
                           mode='max')

earlystop = EarlyStopping(monitor='val_fmeasure', patience=5)

config = Config()
add_compile(model, config)

callback_lists = [checkpointer, reduce]

history = model.fit_generator(generator=training_generator,
                              validation_data=None,#validation_generator,
                              use_multiprocessing=False,
                              epochs=30,
                              verbose=1,
                              callbacks=None)#callback_lists)

(27453,)
(13407,)
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
428/428 [==============================] - 268s 626ms/step - loss: 0.4355 - acc: 0.8911 - fmeasure: 0.4670 - precision: 0.7087
Epoch 2/30
428/428 [==============================] - 226s 527ms/step - loss: 0.2861 - acc: 0.9126 - fmeasure: 0.6096 - precision: 0.7711
Epoch 3/30
428/428 [==============================] - 280s 655ms/step - loss: 0.2590 - acc: 0.9182 - fmeasure: 0.6461 - precision: 0.7791
Epoch 4/30
428/428 [==============================] - 296s 692ms/step - loss: 0.2415 - acc: 0.9241 - fmeasure: 0.6792 - precision: 0.7917
Epoch 5/30
428/428 [==============================] - 298s 697ms/step - loss: 0.2293 - acc: 0.9289 - fmeasure: 0.7003 - precision: 0.8059
Epoch 6/30
428/428 [==============================] - 285s 665ms/step - loss: 0.2204 - acc: 0.9315 - fmeasure: 0.7131 - precision: 0.8143
Epoch 7/30
428/428 [==============================] - 290s 678ms/step - loss: 0.2114 - acc: 0.9341 - fmeas

In [9]:
model.save_weights(model_path + 'densenet_extend_weights-best_one_fold_all_data.hdf5')

In [10]:
def preprocess_y(labels, y, num_class=10):
    bin_label = np.zeros((len(y), num_class)).astype('int8')
    for i in range(len(y)):
        label_nona = labels.loc[y[i]].dropna()
        for j in range(1, label_nona.shape[0]):
            bin_label[i, int(label_nona[j])] = 1
    return bin_label

In [11]:
'''   '''
pre_type = "sym"#"db6"#"sym"

labels = pd.read_csv(path + "REFERENCE.csv")
raw_IDs = labels["File_name"].values.tolist()

IDs = {}
IDs["sym"] = raw_IDs
IDs["db4"] = [i + "_db4" for i in raw_IDs]
IDs["db6"] = [i + "_db6" for i in raw_IDs]

#######################
input_size = (2560, 12)
net_num = 10
inputs_list = [Input(shape=input_size) for _ in range(net_num)]
net = Net()
outputs = net.nnet(inputs_list, 0.5, num_classes=10, attention=False)
model = Model(inputs=inputs_list, outputs=outputs) # without attention and maxpooling and separableconv1d   f=0.819

model_path = './official_densenet_model/'
model_name = "densenet_extend_weights-best_one_fold_all_data.hdf5"#'densenet_extend_weights-best_one_fold_0730.hdf5'

model.load_weights(model_path + model_name)
##########################

X = np.empty((6689, 10, 2560, 12))
for i, ID in enumerate(IDs[pre_type]):
    if i % 1000 == 0:
        print(i)
    X[i,] = np.load("training_data/" + ID + ".npy")
#train_x = [(X[:, i]-np.mean(X[:, i]))/np.std(X[:, i]) for i in range(10)]
train_x = [X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], X[:, 6], X[:, 7], X[:, 8], X[:, 9]]


0
1000
2000
3000
4000
5000
6000


In [12]:
index = np.arange(6689)
y_train = preprocess_y(labels, index)

blend_train = model.predict(train_x)


threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(blend_train.shape[1])

for i in range(blend_train.shape[1]):
    y_prob = np.array(blend_train[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if blend_train[i, j] >= best_threshold[j] else 0 for j in range(blend_train.shape[1])]
          for i in range(len(blend_train))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))

net_num = 10
test_x = [read_data_seg(path, split='Val', preprocess=True, n_index=i, pre_type=pre_type) for i in range(net_num)]

out = model.predict(test_x)
y_pred_test = np.array(
    [[1 if out[i, j] >= best_threshold[j] else 0 for j in range(out.shape[1])] for i in range(len(out))])

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

test_y = y_pred_test

y_pred = [[1 if test_y[i, j] >= best_threshold[j] else 0 for j in range(test_y.shape[1])]
          for i in range(len(test_y))]
pred = []
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(10) if y_pred[j][i] == 1])

val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('jupyter_answers_densenet_{}_0809.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8', 'label9', 'label10'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(10 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

pass

best_threshold : [0.5 0.1 0.1 0.3 0.2 0.2 0.4 0.2 0.4 0.2]
 train data f1_score  : 0.8305535508012308
f1 score of ab 0 is 0.9075643559693143
f1 score of ab 1 is 0.9613049154153642
f1 score of ab 2 is 0.9227571272570245
f1 score of ab 3 is 0.9545073322640414
f1 score of ab 4 is 0.8570014043977652
f1 score of ab 5 is 0.9483618173810205
f1 score of ab 6 is 0.8916374838772433
f1 score of ab 7 is 0.8517114480801631
f1 score of ab 8 is 0.9031904923022938
f1 score of ab 9 is 0.852356753434403
0
0
0
0
0
0
0
0
0
0
